In [3]:
import warmth
from pathlib import Path

In [3]:
1e-6*30e3

0.03

In [2]:
30e-3

0.03

In [ ]:
maps_dir = Path("./data/")

In [ ]:
model = warmth.Model()

In [ ]:
inputs = model.builder.input_horizons_template

In [ ]:
#Add surface grids to the table. You can use other method as well
inputs.loc[0]=[0,"0.gri",None,"Onlap"]
inputs.loc[1]=[66,"66.gri",None,"Onlap"]
inputs.loc[2]=[100,"100.gri",None,"Onlap"]
inputs.loc[3]=[163,"163.gri",None,"Erosive"]
inputs.loc[4]=[168,"168.gri",None,"Erosive"]
inputs.loc[5]=[170,"170.gri",None,"Onlap"]
inputs.loc[6]=[182,"182.gri",None,"Erosive"]
model.builder.input_horizons=inputs
inputs

In [ ]:
inc = 2000
model.builder.define_geometry(maps_dir/"0.gri",xinc=inc,yinc=inc,fformat="irap_binary")

In [ ]:
model.builder.grid.__dict__

In [ ]:
model.builder.extract_nodes(4,maps_dir)

In [ ]:
model.parameters.time_start

In [ ]:
from warmth.data import haq87
model.builder.set_eustatic_sea_level(haq87)

In [ ]:
for i in model.builder.iter_node():
    i.rift=[[182,175]]

In [ ]:
model.simulator.simulate_every = 1

In [ ]:
model.builder.n_valid_node

In [ ]:
model.builder.node

In [ ]:
model.builder.iter_node()

In [4]:
for i in model.builder.node:
    print(i)

NameError: name 'model' is not defined

In [ ]:
%%time
model.simulator.run(save=True,purge=True)

In [ ]:

# interpolate and extrapolate the missing nodes
# find nearby nodes from the array indexer_full_sim, which is sorted by x-index
import itertools
from subsheat3D.fixed_mesh_model import interpolateNode
for ni in range(len(model.builder.nodes)):
    for nj in range(len(model.builder.nodes[ni])):
        if model.builder.nodes[ni][nj] is False:
            closest_x_up = []
            for j in range(ni,len(model.builder.nodes[nj])):
                matching_x = [ i[0] for i in model.builder.indexer_full_sim if i[0]==j ]
                closest_x_up = closest_x_up + list(set(matching_x))
                if len(matching_x)>0:
                    break
            closest_x_down = []
            for j in range(ni-1,-1,-1):
                matching_x = [ i[0] for i in model.builder.indexer_full_sim if i[0]==j ]
                closest_x_down = closest_x_down + list(set(matching_x))
                if len(matching_x)>0:
                    break
            closest_y_up = []
            for j in range(nj,len(model.builder.nodes[ni])):
                matching_y = [ i[1] for i in model.builder.indexer_full_sim if (i[1]==j and ((i[0] in closest_x_up) or i[0] in closest_x_down)) ]
                closest_y_up = closest_y_up + list(set(matching_y))
                if len(matching_y)>0:
                    break
            closest_y_down = []
            for j in range(nj-1,-1,-1):
                matching_y = [ i[1] for i in model.builder.indexer_full_sim if (i[1]==j and (i[0] in closest_x_up or i[0] in closest_x_down) ) ]
                closest_y_down = closest_y_down + list(set(matching_y))
                if len(matching_y)>0:
                    break

            interpolationNodes = [  model.builder.nodes[i[0]][i[1]] for i in itertools.product(closest_x_up+closest_x_down, closest_y_up+closest_y_down)  ]
            interpolationNodes = [nn for nn in interpolationNodes if nn is not False]
            node = interpolateNode(interpolationNodes)
            node.X, node.Y = model.builder.grid.location_grid[ni,nj,:]
            model.builder.nodes[ni][nj] = node
        else:
            node = interpolateNode([model.builder.nodes[ni][nj]])  # "interpolate" the node from itself to make sure the same member variables exist at the end
            model.builder.nodes[ni][nj] = node
        # if (model.builder.nodes[ni][nj].Y>12000):
        #     model.builder.nodes[ni][nj].crustRHP = (2e-6) * 4
        #     model.builder.nodes[ni][nj].rhp = (2e-6) * 4
        model.builder.nodes[ni][nj].crustRHP = (2e0) * 1


In [15]:
from warmth.mesh_model import run
import os
try:
    os.mkdir('mesh')
except FileExistsError:
    pass
try:
    os.mkdir('temp')
except FileExistsError:
    pass
run(model,start_time=model.parameters.time_start,end_time=0)



Rebuild/reload mesh at tti= 182
Using 1D Node parameters NodeParameters1D(shf=0.03, hc=30000.0, hw=3600.0, hLith=130000.0, kLith=3.109, kCrust=2.5, kAsth=100, rhp=2, crustliquid=2500.0, crustsolid=2800.0, lithliquid=2700.0, lithsolid=3300.0, asthliquid=2700.0, asthsolid=3200.0, T0=5, Tm=1330.0, qbase=0.03)
builing
buildVertices


: 